In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [334]:
df = pd.read_csv(r"C:\Users\susha\Downloads\titanic\train.csv")
test_df = pd.read_csv(r"C:\Users\susha\Downloads\titanic\test.csv")

In [335]:
a = pd.get_dummies(df, columns=['Sex','Embarked'], drop_first=True)
b = pd.get_dummies(test_df, columns=['Sex','Embarked'], drop_first=True)

In [336]:
a['Sex_male'] = a['Sex_male'].astype(int)
a['Embarked_Q'] = a['Embarked_Q'].astype(int)
a['Embarked_S'] = a['Embarked_S'].astype(int)
b['Sex_male'] = b['Sex_male'].astype(int)
b['Embarked_Q'] = b['Embarked_Q'].astype(int)
b['Embarked_S'] = b['Embarked_S'].astype(int)

In [337]:
a['FamilySize'] = a['SibSp'] + a['Parch'] + 1
a['IsAlone'] = (a['FamilySize'] == 1).astype(int)
a['HasCabin'] = a['Cabin'].notnull().astype(int)
b['FamilySize'] = b['SibSp'] + test_data['Parch'] + 1
b['IsAlone'] = (b['FamilySize'] == 1).astype(int)
b['HasCabin'] = b['Cabin'].notnull().astype(int)


In [338]:
a['Cabin_letter'] = a['Cabin'].fillna('Missing').apply(lambda x: x[0] if x != 'Missing' else 'M')
a = pd.get_dummies(a, columns=['Cabin_letter'], drop_first=True)
b['Cabin_letter'] = b['Cabin'].fillna('Missing').apply(lambda x: x[0] if x != 'Missing' else 'M')
b = pd.get_dummies(b, columns=['Cabin_letter'], drop_first=True)

In [339]:
a[['Cabin_letter_B','Cabin_letter_C','Cabin_letter_D','Cabin_letter_E','Cabin_letter_F','Cabin_letter_G','Cabin_letter_M','Cabin_letter_T']] = a[['Cabin_letter_B','Cabin_letter_C','Cabin_letter_D','Cabin_letter_E','Cabin_letter_F','Cabin_letter_G','Cabin_letter_M','Cabin_letter_T']].astype(int)

b[['Cabin_letter_B','Cabin_letter_C','Cabin_letter_D','Cabin_letter_E','Cabin_letter_F','Cabin_letter_G','Cabin_letter_M']] = b[['Cabin_letter_B','Cabin_letter_C','Cabin_letter_D','Cabin_letter_E','Cabin_letter_F','Cabin_letter_G','Cabin_letter_M']].astype(int)


In [340]:
combine = [a, b]
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(r' ([A-Za-z]+)\.', expand=False)
    
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')

title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

In [341]:
train_data = a.drop(['PassengerId','Name','Ticket','Cabin','Cabin_letter_T'],axis =1)
test_data = b.drop(['PassengerId','Name','Ticket','Cabin'],axis =1)

In [342]:
train_data['Age'] = train_data.groupby(['Pclass', 'Sex_male'])['Age'].transform(lambda x: x.fillna(x.median()))
test_data['Age'] = test_data.groupby(['Pclass', 'Sex_male'])['Age'].transform(lambda x: x.fillna(x.median()))
test_data['Fare'] = test_data.groupby('Pclass')['Fare'].transform(lambda x: x.fillna(x.median()))

In [343]:
train_data['FareBand'] = pd.qcut(train_data['Fare'], 4, duplicates='drop')
train_data[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)
test_data['FareBand'] = pd.qcut(test_data['Fare'], 4, duplicates='drop')
test_data.groupby('FareBand').mean().sort_values(by='FareBand', ascending=True)
test_data

combine = [train_data, test_data]
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_data = train_data.drop(['FareBand'], axis=1)
test_data = test_data.drop(['FareBand'], axis=1)
combine = [train_data, test_data]


C:\Users\susha\AppData\Local\Temp\ipykernel_9768\2358842890.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_data[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)
C:\Users\susha\AppData\Local\Temp\ipykernel_9768\2358842890.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_data.groupby('FareBand').mean().sort_values(by='FareBand', ascending=True)


In [344]:
train_data = train_data.drop(['Cabin_letter_B','Cabin_letter_C','Cabin_letter_D','Cabin_letter_E','Cabin_letter_F','Cabin_letter_G','Cabin_letter_M'],axis =1)
test_data = test_data.drop(['Cabin_letter_B','Cabin_letter_C','Cabin_letter_D','Cabin_letter_E','Cabin_letter_F','Cabin_letter_G','Cabin_letter_M'],axis =1)

In [350]:
train_data['AgeBand'] = pd.cut(train_data['Age'], 5)
train_data[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

test_data['AgeBand'] = pd.qcut(test_data['Age'], 5, duplicates='drop')
test_data.groupby('AgeBand').mean().sort_values(by='AgeBand', ascending=True)

for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']

train_data

C:\Users\susha\AppData\Local\Temp\ipykernel_9768\1258712119.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train_data[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)
C:\Users\susha\AppData\Local\Temp\ipykernel_9768\1258712119.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  test_data.groupby('AgeBand').mean().sort_values(by='AgeBand', ascending=True)


,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,FamilySize,IsAlone,HasCabin,Title,Age*Class,AgeBand
0,0,3,1.0,1,0,0,1,0,1,2,0,0,1,66.0,"(16.336, 32.252]"
1,1,1,2.0,1,0,3,0,0,0,2,0,1,3,38.0,"(32.252, 48.168]"
2,1,3,1.0,0,0,1,0,0,1,1,1,0,2,78.0,"(16.336, 32.252]"
3,1,1,2.0,1,0,3,0,0,1,2,0,1,3,35.0,"(32.252, 48.168]"
4,0,3,2.0,0,0,1,1,0,1,1,1,0,1,105.0,"(32.252, 48.168]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1.0,0,0,1,1,0,1,1,1,0,5,54.0,"(16.336, 32.252]"
887,1,1,1.0,0,0,2,0,0,1,1,1,1,2,19.0,"(16.336, 32.252]"
888,0,3,1.0,1,2,2,0,0,1,4,0,0,2,64.5,"(16.336, 32.252]"
889,1,1,1.0,0,0,2,1,0,0,1,1,1,1,26.0,"(16.336, 32.252]"


In [351]:
train_data = train_data.drop(['AgeBand'], axis=1)
test_data = test_data.drop(['AgeBand'], axis=1)
combine = [train_data, test_data]

In [352]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

In [353]:
X_train = train_data.drop('Survived', axis=1)
y_train = train_data['Survived']
X_test = test_data

In [355]:
X_train
X_test

,Pclass,Age,SibSp,Parch,Fare,Sex_male,Embarked_Q,Embarked_S,FamilySize,IsAlone,HasCabin,Title,Age*Class
0,3,2.0,0,0,0,1,1,0,1,1,0,1,6.0
1,3,2.0,1,0,0,0,0,1,2,0,0,3,6.0
2,2,3.0,0,0,1,1,1,0,1,1,0,1,6.0
3,3,1.0,0,0,1,1,0,1,1,1,0,1,3.0
4,3,1.0,1,1,1,0,0,1,3,0,0,3,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,3,1.0,0,0,1,1,0,1,1,1,0,1,3.0
414,1,2.0,0,0,3,0,0,0,1,1,1,5,2.0
415,3,2.0,0,0,0,1,0,1,1,1,0,1,6.0
416,3,1.0,0,0,1,1,0,1,1,1,0,1,3.0


In [356]:
cleaned_data = pd.DataFrame(X_train)
cleaned_data.to_csv('C:/Users/susha/Downloads/titanic/titanic_predictions.csv', index=False)

In [357]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=5000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [359]:

submission = pd.DataFrame({
    'PassengerId': b['PassengerId'],
    'Survived': y_pred})
submission.to_csv('C:/Users/susha/Downloads/titanic/titanic_predictions.csv', index=False)


In [314]:
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

submission = pd.DataFrame({
    'PassengerId': b['PassengerId'],
    'Survived': y_pred})
submission.to_csv('C:/Users/susha/Downloads/titanic/titanic_predictionsrf.csv', index=False)


In [188]:
import xgboost as xgb
boost_model = xgb.XGBClassifier(eval_metric='logloss', random_state=42)
boost_model.fit(X_train, y_train)
y_pred = boost_model.predict(X_test)
submission = pd.DataFrame({
    'PassengerId': X_test['PassengerId'],
    'Survived': y_pred})
submission.to_csv('C:/Users/susha/Downloads/titanic/titanic_predictionsxg.csv', index=False)